<img src="../images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"></img>

# Feedback analysis using radiative kernels

---

## Overview

This notebook details all of the steps required to perform a radiative kernel analysis.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Loading CMIP6 data with Intake-ESM](https://projectpythia.org/cmip6-cookbook/notebooks/foundations/intake-esm.html) | Helpful | |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary |  |

- **Time to learn**: 60 minutes

---

## Imports

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
import intake
import s3fs
import fsspec
import glob

## Load in required data

### Climate model output

In this example, we will perform the analysis on a single model from the CMIP6 ensemble, CESM2. The simplest way to calculate feedbacks is to take differences between two climate states, as opposed to regressions. Here we use runs with:
- preindustrial conditions (`piControl`) as the control climate
- instantaneously quadrupled CO$_2$ (`abrupt-4xCO2`) as the perturbed climate

We will use CMIP6 data hosted on Pangeo's Google Cloud Storage:

In [ ]:
cat_url = 'https://storage.googleapis.com/cmip6/pangeo-cmip6.json'
col = intake.open_esm_datastore(cat_url)

The fields (and CMIP names) required to calculate each feedback are:
- Albedo: upwelling and downwelling SW radiation at the surface (`rsus` and `rsds`)
- Temperature (Planck and lapse rate): air temperature (`ta`) and surface temperature (`ts`)
- Water vapor: specific humidity (`hus`) and air temperature
- SW CRE: Net SW radiation at TOA (down `rsdt` minus up `rsut`) and clear-sky versions (down, which is the same, minus up `rsutcs`)
- LW CRE: Net LW radiation at TOA (`rlut`) and the clear-sky version (`rlutcs`)

The cloud feedbacks also require the results from the other feedbacks to correct for noncloud contributions to the CREs.

In [ ]:
cat = col.search(activity_id='CMIP', experiment_id=['piControl', 'abrupt-4xCO2'], table_id='Amon', source_id='CESM2', 
                 variable_id=['rsus', 'rsds', 'ta', 'ts', 'hus', 'rsdt', 'rsut', 'rsutcs', 'rlut', 'rlutcs'])
cat.df

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True})

In [ ]:
ctrl = dset_dict['CMIP.NCAR.CESM2.piControl.Amon.gn']

In [ ]:
ctrl

In [ ]:
pert = dset_dict['CMIP.NCAR.CESM2.abrupt-4xCO2.Amon.gn']

In [ ]:
pert

### Radiative kernels

We will load in radiative kernels from Project Pythia's storage on JetStream2.

In [ ]:
URL = 'https://js2.jetstream-cloud.org:8001/'
path = f'pythia/ClimKern'
fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))
patternKern = f's3://{path}/kernels/ERA5/*.nc'
patternTutorial = f's3://{path}/tutorial_data/*.nc'
filesKern = sorted(fs.glob(patternKern))
filesTutorial = sorted(fs.glob(patternTutorial))
filesetKern = [fs.open(file) for file in filesKern[0:2]]
filesetTutorial = [fs.open(file) for file in filesKern[0:2]]

As we will see in a moment, the naming convention for months as calculated from Xarray's `.groupby()` method ("month", 1-12) is different from what is used in the kernel dataset ("time", 0-11), so we need to align them:

In [ ]:
data_kern = xr.open_dataset(filesetKern[0])
data_kern['time'] = data_kern['time'] + 1
data_kern = data_kern.rename({'time': 'month'})
data_kern

Let's take a look at some of these kernels. First, the albedo kernel, annually averaged:

In [ ]:
data_kern.sw_a.mean(dim='month').plot.contourf(levels=20)

And the LW water vapor kernel, annually and zonally averaged:

In [ ]:
data_kern.lw_q.mean(dim=['month', 'lon']).plot.contourf(levels=40, yincrease=False)

## Prepare data for analysis

Define a function for taking global averages:

In [ ]:
def global_average(data):
    weights = np.cos(np.deg2rad(data.lat))
    data_weighted = data.weighted(weights)
    return data_weighted.mean(dim=['lat', 'lon'], skipna=True)

To get a general idea of how the climate changes in these runs, we can plot the (12-month rolling) global mean surface temperature for the two runs:

In [ ]:
gmst_ctrl = global_average(ctrl.ts.rolling(time=12, center=True).mean())

In [ ]:
gmst_ctrl.sel(time=slice('1150', '1200')).plot()

In [ ]:
gmst_pert = global_average(pert.ts.rolling(time=12, center=True).mean())

In [ ]:
gmst_pert.sel(time=slice('0949', '0999')).plot()

### Calculating the two climate states

Ideally, we would want to compare two equilibrated climates, but since that is usually not possible with standard-length CMIP experiments, we will simply use the monthly climatology over the last 50 years available for each run, which is close enough to equilibrium. The pressure coordinates are also in Pa, so let's convert them to hPa to match the kernels:

In [ ]:
ctrl_state = ctrl.sel(time=slice('1150', '1200')).groupby('time.month').mean(dim='time').squeeze()
pert_state = pert.sel(time=slice('0949', '0999')).groupby('time.month').mean(dim='time').squeeze()

ctrl_state['plev'] = ctrl_state['plev']/100
pert_state['plev'] = pert_state['plev']/100

In [ ]:
pert_state

### Regridding

The model output and kernels are not on the same grid, so we will regrid the kernel dataset to the model's grid using the regridding package `xesmf`. For reusability, let's define a function to regrid data:

In [ ]:
def regrid(ds_in, regrid_to, method='bilinear'):
    regridder = xe.Regridder(ds_in, regrid_to, method=method, periodic=True, ignore_degenerate=True)
    ds_out = regridder(ds_in)
    return ds_out

In [ ]:
regr_kernels = regrid(data_kern, pert_state)

In [ ]:
regr_kernels

Check that the kernels look as expected:

In [ ]:
regr_kernels.sw_a.mean(dim='month').plot.contourf(levels=20)

In [ ]:
regr_kernels.lw_q.mean(dim=['month', 'lon']).plot.contourf(levels=40, yincrease=False)

## Calculate feedbacks

### Surface albedo

### Temperature

#### Planck

#### Lapse rate

### Water vapor

#### Shortwave water vapor

#### Longwave water vapor

### Cloud

#### Cloud radiative effect (CRE)

In [ ]:
ctrl_SWCRE = (-ctrl_state.rsut + ctrl_state.rsutcs)
pert_SWCRE = (-pert_state.rsut + pert_state.rsutcs)
dSWCRE = pert_SWCRE - ctrl_SWCRE

In [ ]:
dSWCRE.plot()

#### Cloud feedback adjustments

### Net

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!